In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical

# Generating data 1000 samples, 10 classes
num_classes = 10
input_shape = (28, 28)  # MNIST image size
x_train = np.random.random((1000, 28, 28))
y_train = np.random.randint(num_classes, size=(1000,))
y_train_cat = to_categorical(y_train, num_classes)

x_test = np.random.random((200, 28, 28))
y_test = np.random.randint(num_classes, size=(200,))
y_test_cat = to_categorical(y_test, num_classes)

In [26]:
# define a simple model function
def create_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Flatten()(inputs)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# initialize main and shadow models
main_model = create_model(input_shape, num_classes)
shadow_model = create_model(input_shape, num_classes)

# x_trusted and y_trusted are trusted datasets for shadow model training
x_trusted = np.random.random((100, 28, 28))
y_trusted = np.random.randint(num_classes, size=(100,))
y_trusted_cat = to_categorical(y_trusted, num_classes)

shadow_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
shadow_model.fit(x_trusted, y_trusted_cat, epochs=5, verbose=1)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0674 - loss: 2.5080  
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2040 - loss: 2.2072 
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2646 - loss: 2.1059 
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3337 - loss: 1.9825 
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3737 - loss: 1.8886 


In [28]:

# Define the function for custom training step
def train_step(model, shadow_model, x_batch, y_batch, lambda_val):
    with tf.GradientTape() as tape:
        y_pred = model(x_batch, training=True)
        primary_loss = tf.keras.losses.categorical_crossentropy(y_batch, y_pred)

        # Consistency check
        y_pred_shadow = shadow_model(x_batch, training=False)
        consistency_loss = tf.reduce_mean(tf.square(y_pred - y_pred_shadow))

        total_loss = primary_loss + lambda_val * consistency_loss  # lambda_val is a weighting factor

    gradients = tape.gradient(total_loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return total_loss

# Training parameters
lambda_val = 0.1  # weighting factor
epochs = 5 
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train_cat)).batch(batch_size)

# custom training step
for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        loss = train_step(main_model, shadow_model, x_batch, y_batch, lambda_val)
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Evaluate the model
test_loss, test_accuracy = main_model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


Epoch 0, Loss: [2.3999588 2.0230422 1.2611344 1.7487859 1.6664709 2.4288967 2.0872529
 1.8408042 1.0994653 2.343625  2.0818322 2.597683  1.7585223 2.182845
 1.5057302 2.2756097 1.6707774 1.8539776 1.8968621 1.3249266 2.0745494
 2.0024781 2.1023252 1.4214364 1.0297443 0.8521562 1.5630612 1.5590552
 1.4611131 1.7211071 1.2645762 1.4769312]
Epoch 0, Loss: [1.8095186 1.7015901 2.4231458 1.6504366 1.2753625 1.1910536 1.5481203
 2.0046003 2.2022493 1.8762941 1.1247044 1.388392  2.4189384 1.8974433
 2.1772614 1.8999145 1.9514008 2.5698977 1.7403939 1.7142639 1.4838998
 0.7894838 2.339999  2.1664047 1.5382621 0.8974124 1.5209126 1.0320987
 2.8242462 2.1791985 1.3795704 1.4880178]
Epoch 0, Loss: [0.9116088 2.5705645 2.1395366 1.3798572 2.1067417 1.6916766 2.2646492
 1.3636851 1.7936976 2.1983926 1.4844986 1.8572812 2.645198  1.5008848
 1.7906036 1.6811056 1.6865712 1.9268713 1.5048214 0.6047311 2.0238924
 1.7338092 1.5781853 2.1047246 1.834081  1.7861402 1.8427148 2.2177413
 1.9245641 1.3371065